In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('itamplify_train.csv')
# drop X and track_id columns
columns_to_drop = ['track_name', 'track_id', 'artists', 'album_name', 'X']
data = data.drop(columns=columns_to_drop, axis=1)
data.head()

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,64,168000,0,0.443,0.778,0,-7.564,0,0.2660,0.241,0.000000,0.2150,0.628,128.250,5,afrobeat
1,44,318360,0,0.580,0.800,6,-7.528,1,0.3450,0.281,0.000000,0.0708,0.483,81.303,4,afrobeat
2,42,171989,0,0.805,0.746,1,-5.211,0,0.1850,0.238,0.000000,0.2150,0.717,127.922,4,afrobeat
3,46,205440,0,0.582,0.660,7,-4.988,0,0.0544,0.399,0.000000,0.1270,0.484,83.789,4,afrobeat
4,44,220800,0,0.776,0.314,2,-9.513,1,0.0340,0.881,0.000723,0.1390,0.476,124.055,4,afrobeat


In [2]:

train, test = train_test_split(data, 
                                stratify=data['track_genre'],
                                test_size=0.2,
                                random_state=42)

train, val = train_test_split(train,
                                stratify=train['track_genre'],
                                test_size=0.2,
                                random_state=42)


train_data = TabularDataset(train)
val_data = TabularDataset(val)
test_data = TabularDataset(test)

label = 'track_genre'
predictor = TabularPredictor(label=label, path='run2',
                             problem_type='multiclass',
                             eval_metric='log_loss',
                             sample_weight='balance_weight',                             
                             ).fit(
                                 use_bag_holdout=True,
                                 train_data = train_data,
                                 tuning_data=val_data,
                                 time_limit=None,
                                 presets='best_quality',
                                 num_cpus=16,
                                 num_gpus=1,
                                 hyperparameters = {
                                'NN_TORCH': {},
                                'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, 'GBMLarge'],
                                'CAT': {},
                                'XGB': {},
                                'FASTAI': {},
                                'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}},
                                       {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}},
                                       {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
                                 'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}},
                                        {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, 
                                        {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
                                'KNN': [{'weights': 'uniform', 'ag_args': {'name_suffix': 'Unif'}},
                                        {'weights': 'distance', 'ag_args': {'name_suffix': 'Dist'}}],
                                 }
                             )

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Using predefined sample weighting strategy: balance_weight. Evaluation metrics will ignore sample weights, specify weight_evaluation=True to instead report weighted metrics.
Beginning AutoGluon training ...
AutoGluon will save models to "run2\"
AutoGluon Version:  0.8.2
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   252.26 GB / 2000.38 GB (12.6%)
Train Data Rows:    5289
Train Data Columns: 15
Tuning Data Rows:    1323
Tuning Data Columns: 15
Label Column: track_genre
Preprocessing data ...
Assigning sample weights to balance differences in frequency of classes.
Train Data Class Count: 15
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    5543.77 MB
	Train Data (Original)  Memory Usage: 0.79 MB (0

	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 2 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 9 | ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', ...]
		('int', [])   : 6 | ['popularity', 'duration_ms', 'explicit', 'key', 'mode', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 9 | ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', ...]
		('int', [])       : 4 | ['popularity', 'duration_ms', 'key', 'time_signature']
		('int', ['bool']) : 2 | ['explicit', 'mode']
	0.1s = Fit runtime
	15 features in original data used to generate 15 feat

In [3]:
predictor.feature_importance(val_data)

Computing feature importance via permutation shuffling for 15 features using 1323 rows with 5 shuffle sets...


	651.6s	= Expected runtime (130.32s per shuffle set)
	149.36s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
popularity,1.059877,0.036269,1.642913e-07,5,1.134555,0.985199
danceability,0.428028,0.032745,4.078200e-06,5,0.495450,0.360607
acousticness,0.356945,0.023878,2.388628e-06,5,0.406109,0.307780
instrumentalness,0.243717,0.008921,2.150251e-07,5,0.262085,0.225349
loudness,0.160046,0.004035,4.843678e-08,5,0.168354,0.151738
valence,0.155889,0.016507,1.486537e-05,5,0.189878,0.121900
speechiness,0.135216,0.006094,4.938848e-07,5,0.147764,0.122667
energy,0.102968,0.008077,4.506098e-06,5,0.119599,0.086338
duration_ms,0.095724,0.012948,3.920575e-05,5,0.122383,0.069064
tempo,0.067534,0.006965,1.338344e-05,5,0.081874,0.053193


In [4]:
predictor.feature_importance(test_data)

Computing feature importance via permutation shuffling for 15 features using 1654 rows with 5 shuffle sets...
	676.38s	= Expected runtime (135.28s per shuffle set)
	173.73s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
popularity,1.078249,0.040743,2.441658e-07,5,1.162139,0.994359
danceability,0.451415,0.007854,1.099397e-08,5,0.467588,0.435243
acousticness,0.344028,0.020292,1.445722e-06,5,0.385809,0.302247
instrumentalness,0.265168,0.015256,1.308965e-06,5,0.296581,0.233756
loudness,0.158458,0.005165,1.352554e-07,5,0.169093,0.147823
valence,0.144274,0.011987,5.665251e-06,5,0.168955,0.119594
speechiness,0.128689,0.007672,1.508632e-06,5,0.144485,0.112892
energy,0.122774,0.005929,6.507998e-07,5,0.134983,0.110566
duration_ms,0.094738,0.014217,5.907719e-05,5,0.124012,0.065464
tempo,0.061310,0.007972,3.354687e-05,5,0.077725,0.044895


In [5]:
predictor.leaderboard(val_data, extra_metrics=['mcc', 'balanced_accuracy'])

                      model  score_test       mcc  balanced_accuracy  score_val  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L3   -1.158658  0.608303           0.599048  -1.161157        7.664349       6.120568  2004.095215                 0.006000                0.001998           1.554154            3       True         26
1       WeightedEnsemble_L2   -1.170300  0.598834           0.591724  -1.170300        2.088804       0.714545   194.345693                 0.243345                0.001508           1.709000            2       True         14
2           CatBoost_BAG_L2   -1.191191  0.593906           0.606335  -1.191191        4.416471       3.124972  1802.521857                 0.130521                0.256089        1562.731459            2       True         20
3         LightGBMXT_BAG_L2   -1.192547  0.587186           0.588053  -1.192547        4.671

,model,score_test,mcc,balanced_accuracy,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-1.158658,0.608303,0.599048,-1.161157,7.664349,6.120568,2004.095215,0.006000,0.001998,1.554154,3,True,26
1,WeightedEnsemble_L2,-1.170300,0.598834,0.591724,-1.170300,2.088804,0.714545,194.345693,0.243345,0.001508,1.709000,2,True,14
2,CatBoost_BAG_L2,-1.191191,0.593906,0.606335,-1.191191,4.416471,3.124972,1802.521857,0.130521,0.256089,1562.731459,2,True,20
3,LightGBMXT_BAG_L2,-1.192547,0.587186,0.588053,-1.192547,4.671782,3.073937,311.732331,0.385832,0.205055,71.941933,2,True,16
4,NeuralNetFastAI_BAG_L2,-1.198348,0.606248,0.592584,-1.198348,4.568454,3.063477,252.359203,0.282503,0.194594,12.568805,2,True,15
5,XGBoost_BAG_L2,-1.199255,0.589227,0.585920,-1.199255,5.479564,3.201483,402.715365,1.193614,0.332600,162.924968,2,True,23
6,CatBoost_BAG_L1,-1.209487,0.584538,0.596907,-1.209487,0.096801,0.100977,126.492340,0.096801,0.100977,126.492340,1,True,8
7,LightGBM_BAG_L2,-1.221982,0.598483,0.593632,-1.221982,4.626506,3.044434,369.328128,0.340555,0.175551,129.537731,2,True,17
8,RandomForestEntr_BAG_L2,-1.223840,0.594300,0.575952,-1.223840,4.509989,3.370017,245.140560,0.224039,0.501135,5.350162,2,True,19
9,LightGBMXT_BAG_L1,-1.226805,0.581751,0.589977,-1.226805,0.412849,0.441607,8.913770,0.412849,0.441607,8.913770,1,True,4


In [6]:
predictor.leaderboard(test_data, extra_metrics=['mcc', 'balanced_accuracy'])

                      model  score_test       mcc  balanced_accuracy  score_val  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L3   -1.195923  0.577820           0.580128  -1.161157        7.964856       6.120568  2004.095215                 0.007002                0.001998           1.554154            3       True         26
1         LightGBMXT_BAG_L2   -1.208024  0.571694           0.586346  -1.192547        4.611202       3.073937   311.732331                 0.243063                0.205055          71.941933            2       True         16
2            XGBoost_BAG_L2   -1.215019  0.571388           0.583461  -1.199255        5.603791       3.201483   402.715365                 1.235652                0.332600         162.924968            2       True         23
3           CatBoost_BAG_L2   -1.219148  0.566628           0.590311  -1.191191        4.501

,model,score_test,mcc,balanced_accuracy,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-1.195923,0.577820,0.580128,-1.161157,7.964856,6.120568,2004.095215,0.007002,0.001998,1.554154,3,True,26
1,LightGBMXT_BAG_L2,-1.208024,0.571694,0.586346,-1.192547,4.611202,3.073937,311.732331,0.243063,0.205055,71.941933,2,True,16
2,XGBoost_BAG_L2,-1.215019,0.571388,0.583461,-1.199255,5.603791,3.201483,402.715365,1.235652,0.332600,162.924968,2,True,23
3,CatBoost_BAG_L2,-1.219148,0.566628,0.590311,-1.191191,4.501677,3.124972,1802.521857,0.133538,0.256089,1562.731459,2,True,20
4,WeightedEnsemble_L2,-1.220706,0.563523,0.567020,-1.170300,1.889218,0.714545,194.345693,0.005000,0.001508,1.709000,2,True,14
5,LightGBM_BAG_L2,-1.247340,0.577371,0.584554,-1.221982,4.538164,3.044434,369.328128,0.170025,0.175551,129.537731,2,True,17
6,CatBoost_BAG_L1,-1.247946,0.551938,0.568111,-1.209487,0.095522,0.100977,126.492340,0.095522,0.100977,126.492340,1,True,8
7,RandomForestEntr_BAG_L2,-1.248833,0.568985,0.566029,-1.223840,4.605248,3.370017,245.140560,0.237109,0.501135,5.350162,2,True,19
8,ExtraTreesGini_BAG_L2,-1.249023,0.573613,0.573380,-1.241051,4.660123,3.565677,241.208117,0.291984,0.696794,1.417719,2,True,21
9,LightGBMXT_BAG_L1,-1.255718,0.543033,0.560416,-1.226805,0.440585,0.441607,8.913770,0.440585,0.441607,8.913770,1,True,4


In [7]:
y_pred = predictor.predict(test_data)
y_pred_proba = predictor.predict_proba(test_data)


In [8]:
predictor.evaluate(test_data)


Evaluation: log_loss on test data: -1.195922975914472
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "log_loss": -1.195922975914472,
    "accuracy": 0.6106408706166868,
    "balanced_accuracy": 0.5801275835300704,
    "mcc": 0.5778200302348073
}


{'log_loss': -1.195922975914472,
 'accuracy': 0.6106408706166868,
 'balanced_accuracy': 0.5801275835300704,
 'mcc': 0.5778200302348073}

In [9]:
predictor.feature_importance(test_data)

Computing feature importance via permutation shuffling for 15 features using 1654 rows with 5 shuffle sets...
	634.78s	= Expected runtime (126.96s per shuffle set)
	169.15s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
popularity,1.078249,0.040743,2.441658e-07,5,1.162139,0.994359
danceability,0.451415,0.007854,1.099397e-08,5,0.467588,0.435243
acousticness,0.344028,0.020292,1.445722e-06,5,0.385809,0.302247
instrumentalness,0.265168,0.015256,1.308965e-06,5,0.296581,0.233756
loudness,0.158458,0.005165,1.352554e-07,5,0.169093,0.147823
valence,0.144274,0.011987,5.665251e-06,5,0.168955,0.119594
speechiness,0.128689,0.007672,1.508632e-06,5,0.144485,0.112892
energy,0.122774,0.005929,6.507998e-07,5,0.134983,0.110566
duration_ms,0.094738,0.014217,5.907719e-05,5,0.124012,0.065464
tempo,0.061310,0.007972,3.354687e-05,5,0.077725,0.044895


In [11]:
from autogluon.tabular import TabularPredictor

# Assume 'predictor' has already been trained and saved to a directory
predictor = TabularPredictor.load('./run2/')  # Load the predictor from where it was saved

# List of specific models to evaluate
models_to_evaluate = ['WeightedEnsemble_L3', 'WeightedEnsemble_L2', 'LightGBMXT_BAG_L2', 'CatBoost_BAG_L2', 'XGBoost_BAG_L2']
# Evaluate each model
for model_name in models_to_evaluate:
    # Get predictions and evaluate specific models
    y_val_pred = predictor.predict(val, model=model_name)
    y_val_true = val['track_genre']  # Ensure your validation data has the correct target column

    # Calculate accuracy per class
    from sklearn.metrics import classification_report
    report = classification_report(y_val_true, y_val_pred, output_dict=True)
    print(f"Accuracy report for {model_name}:")
    print(classification_report(y_val_true, y_val_pred))

Accuracy report for WeightedEnsemble_L3:
              precision    recall  f1-score   support

    afrobeat       0.49      0.44      0.47        54
 alternative       0.41      0.46      0.43        57
       blues       0.49      0.30      0.37        96
   classical       0.57      0.70      0.63        40
     country       0.46      0.49      0.47        47
       disco       0.60      0.59      0.59       111
        folk       0.57      0.58      0.57       123
        funk       0.81      0.72      0.76       101
 heavy-metal       0.71      0.85      0.78        67
     hip-hop       0.58      0.76      0.66        92
        jazz       0.36      0.21      0.26        43
  psych-rock       0.61      0.69      0.65       137
   reggaeton       0.64      0.57      0.60        60
       samba       0.81      0.87      0.84       156
      techno       0.83      0.76      0.79       139

    accuracy                           0.64      1323
   macro avg       0.60      0.60      

In [15]:
your_model_object = predictor._trainer.load_model('WeightedEnsemble_L3')
your_model_object

In [19]:
import pickle
import os 

# Load AutoGluon predictor
predictor = TabularPredictor.load('run2/')  # Adjust path as necessary

# List of model names you want to save
model_names = ['WeightedEnsemble_L3', 'WeightedEnsemble_L2', 'LightGBMXT_BAG_L2', 'CatBoost_BAG_L2', 'XGBoost_BAG_L2']

# Create directory for saving models if it does not already exist
model_dir = './best_models'
os.makedirs(model_dir, exist_ok=True)

# Save each model as a pickle file
for model_name in model_names:
    # Load the model object
    model = predictor._trainer.load_model(model_name)
    
    # Save the model to a pickle file
    model_path = os.path.join(model_dir, f'{model_name}.pkl')
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)
    print(f"Saved {model_name} to {model_path}")

Saved WeightedEnsemble_L3 to ./best_models\WeightedEnsemble_L3.pkl
Saved WeightedEnsemble_L2 to ./best_models\WeightedEnsemble_L2.pkl
Saved LightGBMXT_BAG_L2 to ./best_models\LightGBMXT_BAG_L2.pkl
Saved CatBoost_BAG_L2 to ./best_models\CatBoost_BAG_L2.pkl
Saved XGBoost_BAG_L2 to ./best_models\XGBoost_BAG_L2.pkl


In [20]:
import pickle
import os

# Directory where the models are saved
model_dir = './best_models'
model_names = ['WeightedEnsemble_L3', 'WeightedEnsemble_L2', 'LightGBMXT_BAG_L2', 'CatBoost_BAG_L2', 'XGBoost_BAG_L2']

# Load the models
models = {}
for model_name in model_names:
    model_path = os.path.join(model_dir, f'{model_name}.pkl')
    with open(model_path, 'rb') as f:
        models[model_name] = pickle.load(f)

# Prepare your test data
# Assume 'test' is already loaded and preprocessed similarly to your training data

# Making predictions with each model
for model_name, model in models.items():
    y_test_pred = model.predict(test.drop(columns=['track_genre']))
    # Evaluate the predictions
    # Here you can calculate the accuracy or other metrics
    from sklearn.metrics import accuracy_score
    y_test_true = test['track_genre']  # Make sure this is your target column
    accuracy = accuracy_score(y_test_true, y_test_pred)
    print(f"Accuracy for {model_name}: {accuracy:.4f}")


Accuracy for WeightedEnsemble_L3: 0.0000
Accuracy for WeightedEnsemble_L2: 0.0000
Accuracy for LightGBMXT_BAG_L2: 0.0000
Accuracy for CatBoost_BAG_L2: 0.0000
Accuracy for XGBoost_BAG_L2: 0.0000
